# VCF file from MIS
##fileformat=VCFv4.1
##filedate=2018.3.17
##source=Minimac3
##contig=<ID=15>
##FILTER=<ID=GENOTYPED,Description="Marker was genotyped AND imputed">
##FILTER=<ID=GENOTYPED_ONLY,Description="Marker was genotyped but NOT imputed">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=DS,Number=1,Type=Float,Description="Estimated Alternate Allele Dosage : [P(0/1)+2*P(1/1)]">
##FORMAT=<ID=GP,Number=3,Type=Float,Description="Estimated Posterior Probabilities for Genotypes 0/0, 0/1 and 1/1 ">
##INFO=<ID=AF,Number=1,Type=Float,Description="Estimated Alternate Allele Frequency">
##INFO=<ID=MAF,Number=1,Type=Float,Description="Estimated Minor Allele Frequency">
##INFO=<ID=R2,Number=1,Type=Float,Description="Estimated Imputation Accuracy">
##INFO=<ID=ER2,Number=1,Type=Float,Description="Empirical (Leave-One-Out) R-square (available only for genotyped variants)
">
#CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  EMERGE-1_16224699       EMERGE-1_16224832       EM
ERGE-1_16225552 EMERGE-1_16225624       EMERGE-1_16226154       EMERGE-1_16233448       EMERGE-2_16229819       EMERGE-3_1
6218877 EMERGE-3_16219796       EMERGE-3_16220543       EMERGE-3_16220698       EMERGE-3_16220967       EMERGE-3_16221838EMERGE-3_16222896        EMERGE-3_16223176       EMERGE-3_16223472       EMERGE-3_16223514       EMERGE-3_16223639       EM
ERGE-3_16225243 EMERGE-3_16225590       EMERGE-3_16225928       EMERGE-3_16226294       EMERGE-3_16228484       EMERGE-3_1
6228513 EMERGE-3_16229771       EMERGE-3_16229817       EMERGE-3_16229912       EMERGE-3_16230463       EMERGE-3_16230480EMERGE-3_16230920        EMERGE-3_16232087       EMERGE-3_16232393       EMERGE-3_16232421       EMERGE-3_16232533       EM
ERGE-3_16233580 EMERGE-3_16234445       EMERGE-3_16234448       EMERGE-4_16226213       EMERGE-4_16226273       EMERGE-5_1
6223236 EMERGE-6_16221489       EMERGE-7_16220844       EMERGE-8_16220003       EMERGE-9_16217786       EMERGE-9_16231695EMERGE-10_16230033       EMERGE-10_16230638      EMERGE-11_16214874      EMERGE-11_16215381      EMERGE-11_16215457  

---



**rs16969968 gwas results**
rs16969968 15 78882925 G A 0.34688 0.34688 0.997 0.98853 730 0.352004 0.0525009 0.0338811 2.42038 2.4011455861625 0.121246432020861 1.05(0.99-1.13)


## info file
SNP     REF(0)  ALT(1)  ALT_Frq MAF     AvgCall Rsq     Genotyped       LooRsq  EmpR    EmpRsq  Dose0   Dose1
15:20000041     T       A       0.00594 0.00594 0.99406 0.00013 Imputed -       -       -       -       -
15:20000075     C       T       0.01359 0.01359 0.98641 0.00019 Imputed -       -       -       -       -
15:20000114     G       T       0.00998 0.00998 0.99002 0.00040 Imputed -       -       -       -       -
15:20000123     T       C       0.00113 0.00113 0.99887 0.00001 Imputed -       -       -       -       -
15:20000127     A       T       0.00363 0.00363 0.99637 0.00006 Imputed -       -       -       -       -




# Filter by P-value

In [ ]:
## EC2 console ##
cd /shared/s3/emerge/data/assoc_tests/002/final

ls 

"""emerge_ea.1000G.chr10.CAT_FTND~SNP+SEX+EVs.subject_1000g_maf_gt_0.01_rsq_gt_0.30_ea.gz
...
"""

outFile=emerge.ea.1000G_p3.CAT_FTND~SNP+SEX+EVs.maf_gt_0.01.rsq_gt_0.30.p_lte_0.001
zcat emerge_ea.1000G.chr10.CAT_FTND~SNP+SEX+EVs.subject_1000g_maf_gt_0.01_rsq_gt_0.30_ea.gz |\
    head > $ouFile
for file in *; do
    echo Processing $file
    zcat $file | tail -n +2 |
    perl -lane 'if ($F[15] <= 0.001) { print; }' >> \
    $outFile
done

In [ ]:
### START Filter by p-value ###

# MAF > 0.01 in AFR and EUR
#for ancestry in aa; do
  if [ $ancestry == "aa" ]; then
    group=afr
  else
    group=eur
  fi
  for ext in $group ${group}+$COHORT.rsq_gt_0.3; do
    outFile=$ASSOCIATION_ROOT/$ancestry/processing/$COHORT.$ancestry.1000G_p3.$MODEL.maf_gt_0.01_$ext.p_lte_0.001
    head -n 1 $ASSOCIATION_ROOT/$ancestry/final/$COHORT.$ancestry.1000G_p3.chr1.$MODEL.maf_gt_0.01_$ext > \
      $outFile
    for (( chr=1; chr<24; chr++ )); do
      echo Processing ${ASSOCIATION_ROOT}/$ancestry/final/$COHORT.$ancestry.1000G_p3.chr$chr.$MODEL.maf_gt_0.01_$ext
      tail -n +2 $ASSOCIATION_ROOT/$ancestry/final/$COHORT.$ancestry.1000G_p3.chr$chr.$MODEL.maf_gt_0.01_$ext |
        perl -lane 'if ($F[15] <= 0.001) { print; }' >> \
        $outFile
    done
done
#done

# Sort
#for (ancestry in c("ea","aa")){
for (ancestry in c("aa")){
        cohort="copdgene2"
        if (ancestry == "aa") { group = "afr" } else if (ancestry == "ea") { group = "eur" }
        dat=read.table(paste0('/share/nas04/jmarks/gwas/',cohort, '/association_tests/001/',
                              ancestry,'/processing/', cohort, '.',ancestry,
                              '.1000G_p3.CAT_FTND~SNP+AGE+SEX+GOLD+EVs.maf_gt_0.01_',
                              group,'.p_lte_0.001'), header = TRUE)
        dat = dat[order(dat$p),]
        write.csv(dat,
            file = paste0('/share/nas04/jmarks/gwas/',cohort,'/association_tests/001/',
                          ancestry,'/final/', cohort, '.',ancestry,'.1000G_p3.CAT_FTND~SNP+AGE+SEX+GOLD+EVs.maf_gt_0.01_',
                          group,'.p_lte_0.001.csv'), row.names = FALSE)


        dat=read.table(paste0('/share/nas04/jmarks/gwas/', cohort,'/association_tests/001/',
                              ancestry,'/processing/', cohort, '.',ancestry,
                              '.1000G_p3.CAT_FTND~SNP+AGE+SEX+GOLD+EVs.maf_gt_0.01_',group,
                              '+', cohort, '.rsq_gt_0.3.p_lte_0.001'), header=TRUE)
        dat = dat[order(dat$p),]
        write.csv(dat,
                  file = paste0('/share/nas04/jmarks/gwas/', cohort, '/association_tests/001/',
                         ancestry,'/final/', cohort, '.', ancestry, '.1000G_p3.CAT_FTND~SNP+AGE+SEX+GOLD+EVs.maf_gt_0.01_',
                         group,'+',cohort,'.rsq_gt_0.3.p_lte_0.001.csv'), row.names = FALSE)
}

## END Filter by p-value ###
